In [ ]:
# !sudo apt install unzip
!gdown 1ulMESAGvffNUOGK9sTsrt2ELEp8y5wmi
!unzip saved_models.zip

In [2]:

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import Adafactor
from datasets import load_dataset
from datetime import datetime
import pandas as pd
import random
import string
import torch

In [3]:
def generate_seeds(seed_name: str, n: int):
    seeds = []
    if seed_name == 'time':
        max_source_length = 64
        for x in range(n):
            seeds.append(datetime.now().strftime("%Y:%m:%d %H:%M:%S:%f") + " " + str(random.randint(0, 100_000_000_000_00)))
        
        prefix = "Generate a FHIR data row using the seed: "
        seeds = {
            "seed": [prefix + doc for doc in seeds]
        }
        
        return seeds, max_source_length

    elif seed_name == 'random string':
        max_source_length = 64
        for x in range(n):
            seeds.append(''.join(random.SystemRandom().choice(string.ascii_uppercase + string.digits) for _ in range(128)))
        prefix = "Generate a FHIR data row using the seed: "
        seeds = {
            "seed": [prefix + doc for doc in seeds]
        }
        
        return seeds, max_source_length

    else:
        raise ValueError(f'Unknown seed name: {seed_name}')

In [4]:
#generate_seeds('time', 5)
#generate_seeds('random string', 5)

({'seed': ['Generate a FHIR data row using the seed: 2023:08:31 10:49:20:056875 6150673943501',
   'Generate a FHIR data row using the seed: 2023:08:31 10:49:20:056875 1101172712791',
   'Generate a FHIR data row using the seed: 2023:08:31 10:49:20:056875 7525435332086',
   'Generate a FHIR data row using the seed: 2023:08:31 10:49:20:056875 2691349309467',
   'Generate a FHIR data row using the seed: 2023:08:31 10:49:20:056875 7223912987934']},
 64)

In [6]:
seeds, max_source_length = generate_seeds('time', 5)

In [8]:
saved_model_dir = "./saved_models"
tokenizer       = T5Tokenizer.from_pretrained(saved_model_dir, legace=False)
model           = T5ForConditionalGeneration.from_pretrained(saved_model_dir).to('cuda')

In [ ]:
model_outputs = model.generate(**
    tokenizer(
        text_target      = seeds['seed'],
        padding          = "longest",
        max_length       = max_source_length,
        truncation       = True,
        return_tensors    = 'pt'
    ).to('cuda'),
    max_length=4096
)

In [ ]:
df = pd.DataFrame({
    "seed": seeds['seed'],
    "Generated_FHIR": tokenizer.batch_decode(model_outputs, skip_special_tokens=True)
    })

In [ ]:
df.to_csv("generated_fhir.csv", index=False)